## Closed Loop Simulation Notebook

This notebook breaks down the closed loop simulation in different jupyter notebook cells. The aim of this notebook is to ease the debugging and development of the different packages involved in the Optical Feedback Control loop.

This notebook was last run on July 7th 2022. 
Details:
- Stack release: 2021_25, Last verified to run 2022-07-07
- ts_phosim release: v2.0.4


### Required python packages

In [1]:
import argparse
import logging
import numpy as np
import os
import shutil
import subprocess
import sys
import time
import batoid

from lsst.ts.batoid.CloseLoopTask import CloseLoopTask
from lsst.ts.batoid.utils.PlotUtil import plotFwhmOfIters
from lsst.daf import butler as dafButler
from lsst.ts.wep.Utility import CamType, FilterType, runProgram
from lsst.ts.batoid.utils.Utility import getPhoSimPath, getAoclcOutputPath, getCamera
from lsst.ts.batoid.utils.SensorWavefrontError import SensorWavefrontError
from lsst.ts.batoid.wfsim.wfsim import SSTBuilder

from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
import matplotlib.pyplot as plt

In [2]:
%%time

# Set the parser
parser = argparse.ArgumentParser(
    description="Run AOS closed-loop simulation (default is amplifier files)."
)
parser = CloseLoopTask.setDefaultParser(parser)
parser = CloseLoopTask.setImgParser(parser)

# Get the default arguments
sys.argv = ['-f']
args = parser.parse_args()

# Print default arguments
print(args)

logger = logging.getLogger()
logger.setLevel(args.log_level)

# Initialize the ClosedLoopTask
closeLoopTask = CloseLoopTask()

Namespace(inst='comcam', filterType='', rotCam=0.0, m1m3FErr=0.05, numOfProc=1, iterNum=5, output='', log_level=20, clobber=False, pipelineFile='', boresightDeg=[0, 0], skyFile='', eimage=False)
CPU times: user 4.01 ms, sys: 1.82 ms, total: 5.83 ms
Wall time: 4.77 ms


In [3]:
PHOSIMPATH = "/home/gmegias/jhome/aos/phosim_syseng4/"
AOCLCOUTPUTPATH = "/home/gmegias/jhome/aos/ts_batoid/output/"
os.environ["PHOSIMPATH"] = PHOSIMPATH
os.environ["AOCLCOUTPUTPATH"] = AOCLCOUTPUTPATH

args.inst = 'comcam' 
args.boresightDeg = [0.03, -0.02]
args.eimage = None
args.skyFile = '/home/gmegias/jhome/aos/ts_batoid/tests/testData/sky/skyComCam.txt'
args.output = '/home/gmegias/jhome/aos/perturbations/test2253/'a
args.filterType = FilterType.REF

if os.path.exists(args.output):
    shutil.rmtree(args.output)

In [4]:
boresight = [0, 0]
rotCamInDeg = args.rotCam
rotAngInDeg = 0.0
useEimg = args.eimage
m1m3ForceError = args.m1m3FErr
numPro = args.numOfProc
iterNum = args.iterNum
doErsDirCont = args.clobber
pathSkyFile = args.skyFile

In [5]:
# Check the input arguments
camType, instName = closeLoopTask.getCamTypeAndInstName(args.inst)
filterType = closeLoopTask.getFilterType(args.filterType)
baseOutputDir = closeLoopTask.checkAndCreateBaseOutputDir(args.output)

# Config feedback control loop
closeLoopTask.configOfcCalc(instName)

# Config Batoid
closeLoopTask.configBatoidCmpt(
        instName,
        filterType,
        rotAngInDeg,
        m1m3ForceError,
        numPro,
        boresight=boresight,
        zAngleInDeg=27.0912,
        seedNum=6,
    )

In [6]:
# Set the telescope state to be the same as the OFC
dofInUm = closeLoopTask.ofcCalc.ofc_controller.aggregated_state
closeLoopTask.batoidCmpt.setDofInUm(dofInUm)

# Get the list of referenced sensor name (field positions)

# If using wavefront sensors we measure one per pair
# and the field
if camType == CamType.LsstCam:
    cornerSensorNameList = closeLoopTask.getSensorNameListOfFields(instName)
    cornerSensorIdList = closeLoopTask.getSensorIdListOfFields(instName)
    refSensorNameList = []
    refSensorIdList = []
    for name, id in zip(cornerSensorNameList, cornerSensorIdList):
        if name.endswith("SW0"):
            refSensorNameList.append(name)
            refSensorIdList.append(id)
else:
    refSensorNameList = closeLoopTask.getSensorNameListOfFields(instName)
    refSensorLocationList = closeLoopTask.getSensorLocationListOfFields(instName)
    refSensorIdList = closeLoopTask.getSensorIdListOfFields(instName)


# Common file and directory names
opdZkFileName = "opd.zer"
opdPssnFileName = "PSSN.txt"
outputDirName = "pert"
outputImgDirName = "img"
iterDefaultDirName = "iter"
dofInUmFileName = "dofPertInNextIter.mat"
fwhmItersFileName = "fwhmIters.png"
if args.pipelineFile == "":
    pipelineFile = None

# Specific file names to the amplifier/eimage
wfsZkFileName = "wfs.zer"

In [ ]:
iterCount = 0
obsId = 9006000

# Set the observation Id
closeLoopTask.phosimCmpt.setSurveyParam(obsId=obsId)

# The iteration directory
iterDirName = "%s%d" % (iterDefaultDirName, iterCount)

outputDir = os.path.join(baseOutputDir, iterDirName, outputDirName)
closeLoopTask.batoidCmpt.setOutputDir(outputDir)

outputDir = os.path.join(baseOutputDir, iterDirName, outputImgDirName)
closeLoopTask.batoidCmpt.setOutputDir(outputDir)

In [7]:
listOfWfErr = closeLoopTask.batoidCmpt.runBatoid(
    obsId, refSensorIdList, refSensorLocationList
)

In [8]:
builder = builder.with_aos_dof(dofInUm)
optic = builder.build()

iterCount += 1
outputDirName = "pert"
outputImgDirName = "img"
iterDefaultDirName = "iter"
baseOutputDir = closeLoopTask.checkAndCreateBaseOutputDir(args.output)
# The iteration directory
iterDirName = "%s%d" % (iterDefaultDirName, iterCount)

outputDir = os.path.join(baseOutputDir, iterDirName, outputDirName)
closeLoopTask.phosimCmpt.setOutputDir(outputDir)

outputDir = os.path.join(baseOutputDir, iterDirName, outputImgDirName)
closeLoopTask.phosimCmpt.setOutputDir(outputDir)

NameError: name 'builder' is not defined

In [15]:
outputImgDirName = "img"
outputImgDir = os.path.join(baseOutputDir, iterDirName, outputImgDirName)

closeLoopTask.batoidCmpt.metr.setWgtAndFieldXyOfGQ(instName)
closeLoopTask.batoidCmpt.setOutputImgDir(outputImgDir)

closeLoopTask.phosimCmpt.analyzeOpdData(
    instName,
    zkFileName=opdZkFileName,
    rotOpdInDeg=-rotCamInDeg,
    pssnFileName=opdPssnFileName,
)


In [11]:
# Get the PSSN from file
pssn = closeLoopTask.batoidCmpt.getOpdPssnFromFile(opdPssnFileName)
print("Calculated PSSN is %s." % pssn)

# Get the GQ effective FWHM from file
gqEffFwhm = closeLoopTask.batoidCmpt.getOpdGqEffFwhmFromFile(opdPssnFileName)
print("GQ effective FWHM is %.4f." % gqEffFwhm)

# Set the FWHM data
print("Setting FWHM data...")
t0 = time.time()
fwhm, sensor_id = closeLoopTask.batoidCmpt.getListOfFwhmSensorData(
    opdPssnFileName, refSensorIdList
)

closeLoopTask.ofcCalc.set_fwhm_data(fwhm, sensor_id)
ttl_time = time.time() - t0
print("FWHM data set, took {} seconds.".format(ttl_time))

Calculated PSSN is [0.5861822  0.58574083 0.59020358 0.58073176 0.57745064 0.58226992
 0.58156714 0.57865639 0.58104174].
GQ effective FWHM is 0.5515.
Setting FWHM data...
FWHM data set, took 0.0016791820526123047 seconds.


In [13]:
# Record the wavefront error with the same order as OPD for the
# comparison


# Calculate the DOF
print('Calculating DOF...')
t0 = time.time()
wfe = np.array(
    [sensor_wfe.getAnnularZernikePoly() for sensor_wfe in listOfWfErr]
)
sensor_ids = np.array(
    [sensor_wfe.getSensorId() for sensor_wfe in listOfWfErr]
)
ttl_time = time.time() - t0
print('DOF computed. Took {} seconds.'.format(ttl_time))

print('Calculating corrections...')
t0 = time.time()
closeLoopTask.ofcCalc.calculate_corrections(
    wfe=wfe,
    field_idx=sensor_ids,
    filter_name=str('R'),
    gain=-1,
    rot=rotAngInDeg,
)
ttl_time = time.time() - t0
print('Corrections computed. Took {} seconds.'.format(ttl_time))

# Set the new aggregated DOF to phosimCmpt
print('Setting and saving new DOF...')
t0 = time.time()
dofInUm = closeLoopTask.ofcCalc.ofc_controller.aggregated_state
closeLoopTask.batoidCmpt.setDofInUm(dofInUm)

# Save the DOF file
closeLoopTask.batoidCmpt.saveDofInUmFileForNextIter(
    dofInUm
)
ttl_time = time.time() - t0
print('New DOF saved. Took {} seconds.'.format(ttl_time))

# Add the observation ID by 10 for the next iteration
obsId += 10

Calculating DOF...
DOF computed. Took 0.0004107952117919922 seconds.
Calculating corrections...
Corrections computed. Took 0.013369083404541016 seconds.
Setting and saving new DOF...
New DOF saved. Took 0.0019941329956054688 seconds.


The wavefront error of the first simulation iteration is printed below

### Summarize the FWHM and plot results

In [29]:
# Summarize the FWHM
fwhmItersFileName = "fwhmIters.png"
pssnFiles = [
    os.path.join(
        baseOutputDir,
        "%s%d" % (iterDefaultDirName, num),
        outputImgDirName,
        opdPssnFileName,
    )
    for num in [0,1,2,3]
]
saveToFilePath = os.path.join(baseOutputDir, fwhmItersFileName)
plotFwhmOfIters(pssnFiles, saveToFilePath=saveToFilePath)

In [30]:
# importing Image class from PIL package
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

im = np.array(Image.open(saveToFilePath))
plt.figure(figsize=(10,10))
plt.imshow(im)
plt.axis('off')
plt.show()


/tmp/ipykernel_1804689/2572018048.py:10: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [ ]:
for idx in [0]:

    image_file = get_pkg_data_filename('/home/gmegias/jhome/aos/perturbations/test2253/iter0/img/opd_9006000_4.fits.gz')
    image_data = fits.getdata(image_file, ext=0)
    masked =np.ma.masked_where(image_data == 0, image_data)

    image_file = get_pkg_data_filename('/home/gmegias/jhome/aos/perturbations/testNe3/iter0/img/opd_9006000_4.fits.gz')
    image_data2 = fits.getdata(image_file, ext=0)
    masked2 =np.ma.masked_where(image_data2 == 0, image_data2)


    print(np.min(image_data))
    print(np.max(image_data))

    plt.figure(figsize = (10,9))
    plt.imshow(abs(masked - masked2))
    plt.colorbar()
    plt.savefig('/home/gmegias/jhome/aos/perturbations/trial/diff_imagsse0{}s.png'.format(idx))
    plt.close()